In [211]:
import numpy as np
from astropy.io import fits
import os.path
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
%matplotlib inline

In [216]:
# eventlist = "/Users/abigailstevens/Reduced_data/GX339-BQPO/95409-01-17-06/eventlist_1.fits"
eventlist = "/Users/abigailstevens/Dropbox/Academic/Conferences\ \&\ Talks/DC_talks/cygx1_counts.lc"
time_binning = .02  # seconds
lc_length = 2  # seconds
show_fits_info = False
pcu = 2  # -1 = all
channel = -1   # -1 = all

In [217]:
detchans = 64
if not os.path.isfile(eventlist):
    raise Exception("ERROR: Event list does not exist: %s" % eventlist)
assert pcu <= 4, "PCU must be between 0 and 4 inclusive, or -1 for all PCUs."

try:
    fits_hdu = fits.open(eventlist)
except IOError:
    print "Issue opening fits file event list: %s" % eventlist

header = fits_hdu[0].header
data = fits_hdu[1].data
fits_hdu.close()

if show_fits_info:
    print header.keys
    print "\n", data.columns.names
    
if pcu != -1:
    PCU2_mask = data.field('PCUID') != pcu
    data = data[PCU2_mask]
# if channel != -1:
#     channel_mask = data.field('CHANNEL') == channel
#     data = data[channel_mask]
# print len(data)

In [220]:
absolute_start_time = data.field('TIME')[0]
time = np.asarray(data.field('TIME')) - absolute_start_time
chan = np.asarray(data.field('CHANNEL'))
# start_time = time[0]
start_time = 10
end_time = time[-1]
if lc_length > end_time:
    print "Requested lightcurve length is longer than data. Making light curve of whole data set."
    lc_length = end_time

time_edges = np.arange(start_time, start_time+lc_length+time_binning, time_binning)
chan_edges = np.arange(0,detchans+1, 1)
lightcurve, t_edges, c_edges = np.histogram2d(time, chan, bins=[time_edges, chan_edges])
t_bin_centers = 0.5 * (t_edges[1:]+t_edges[:-1])
mean_count_rate = np.sum(lightcurve, axis=0) / lc_length

In [229]:
if channel == -1:
    lc = np.sum(lightcurve[:,2:26], axis=1)
    title = "Reference band"
    title = ""
    mean_rate = np.sum(mean_count_rate[2:26])
else:
    lc = lightcurve[:,channel]
    title = "Channel %d" % channel
    mean_rate = mean_count_rate[channel]

print "Mean count rate:", mean_rate

font_prop = font_manager.FontProperties(size=18)

fig, ax = plt.subplots(1, 1, figsize=(16, 6), dpi=300)
ax.plot(t_bin_centers-start_time, lc, lw=3, color='purple')
ax.set_title(title)
ax.set_xlabel("Arbitrary time (s)", fontproperties=font_prop)
ax.set_ylabel("Photon counts", fontproperties=font_prop)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.set_ylim(5, 23)
# plt.show()
plt.savefig("/Users/abigailstevens/Dropbox/Academic/95409-01-17-06_ref_lc.eps")
plt.close()

Mean count rate: 702.5
